In [ ]:
## Query information_schema with SELECT

# 1
# -- Query the right table in information_schema
SELECT table_name 
FROM information_schema.tables
-- Specify the correct table_schema value
WHERE table_schema = 'public';

# 2
# -- Query the right table in information_schema to get columns
SELECT column_name, data_type 
FROM information_schema.columns 
WHERE table_name = 'university_professors' AND table_schema = 'public';

# 3
# -- Query the first five rows of our table
SELECT * 
FROM university_professors 
LIMIT 5;

In [ ]:
## CREATE your first few TABLEs

# 1
# -- Create a table for the professors entity type
CREATE TABLE professors (
 firstname text,
 lastname text
);

-- Print the contents of this table
SELECT * 
FROM professors

# 2
# -- Create a table for the universities entity type
CREATE TABLE universities (
        university_shortname text,
        university text,
        university_city text
);

# -- Print the contents of this table
SELECT * 
FROM universities

In [ ]:
## ADD a COLUMN with ALTER TABLE

# -- Add the university_shortname column
ALTER TABLE professors
ADD COLUMN university_shortname text;

# -- Print the contents of this table
SELECT * 
FROM professors

In [ ]:
## RENAME and DROP COLUMNS in affiliations

## -- Rename the organisation column
ALTER TABLE affiliations
RENAME COLUMN organisation TO organization;

## -- Delete the university_shortname column
ALTER TABLE affiliations
DROP COLUMN university_shortname;

In [ ]:
## Migrate data with INSERT INTO SELECT DISTINCT

# 1
## -- Insert unique professors into the new table
INSERT INTO professors 
SELECT DISTINCT firstname, lastname, university_shortname 
FROM university_professors;

## -- Doublecheck the contents of professors
SELECT * 
FROM professors;


# 2
## -- Insert unique affiliations into the new table
INSERT INTO affiliations 
SELECT DISTINCT firstname, lastname, function, organization 
FROM university_professors;

## -- Doublecheck the contents of affiliations
SELECT * 
FROM affiliations;

In [ ]:
## Delete tables with DROP TABLE

## -- Delete the university_professors table
DROP TABLE university_professors;

In [ ]:
## Conforming with data types

## -- Let's add a record to the table
INSERT INTO transactions (transaction_date, amount, fee) 
VALUES ('2018-09-24', 5454, '30');

## -- Doublecheck the contents
SELECT *
FROM transactions;

In [ ]:
## Type CASTs

# -- Calculate the net amount as amount + fee
SELECT transaction_date, amount + CAST(fee as INTEGER) AS net_amount 
FROM transactions;

In [ ]:
## Change types with ALTER COLUMN

# -- Select the university_shortname column
SELECT DISTINCT(university_shortname) 
FROM professors;

# -- Specify the correct fixed-length character type
ALTER TABLE professors
ALTER COLUMN university_shortname
TYPE char(3);

# -- Change the type of firstname
ALTER TABLE professors
ALTER COLUMN firstname
TYPE varchar(64);

In [ ]:
## Convert types USING a function

# -- Convert the values in firstname to a max. of 16 characters
ALTER TABLE professors 
ALTER COLUMN firstname 
TYPE varchar(16)
USING SUBSTRING(firstname FROM 1 FOR 16);

In [ ]:
## Disallow NULL values with SET NOT NULL

# -- Disallow NULL values in firstname
ALTER TABLE professors 
ALTER COLUMN firstname SET NOT NULL;

# -- Disallow NULL values in lastname
ALTER TABLE professors
ALTER COLUMN lastname SET NOT NULL

In [ ]:
## What happens if you try to enter NULLs?

INSERT INTO professors ( firstname, lastname, university_shortname)
VALUES (NULL, "Miller", "ETH");

In [ ]:
## Make your columns UNIQUE with ADD CONSTRAINT

# -- Make universities.university_shortname unique
ALTER TABLE universities
ADD CONSTRAINT university_shortname_unq UNIQUE(university_shortname);


-- Make organizations.organization unique
ALTER TABLE organizations
ADD CONSTRAINT organization_unq UNIQUE(organization)

In [ ]:
## Get to know SELECT COUNT DISTINCT

# -- Count the number of rows in universities
SELECT COUNT(*) 
FROM universities;


# -- Count the number of distinct values in the university_city column
SELECT COUNT(DISTINCT(university_city)) 
FROM universities;

In [ ]:
## Identify keys with SELECT COUNT DISTINCT

# -- Try out different combinations
SELECT COUNT(DISTINCT(firstname, lastname)) 
FROM professors;

In [ ]:
## ADD key CONSTRAINTs to the tables

# -- Rename the organization column to id
ALTER TABLE organizations
RENAME COLUMN organization TO id;

# -- Make id a primary key
ALTER TABLE organizations
ADD CONSTRAINT organization_pk PRIMARY KEY (id);
__________________________________________________

# -- Rename the university_shortname column to id
ALTER TABLE universities
RENAME COLUMN university_shortname TO id;

# -- Make id a primary key
ALTER TABLE universities
ADD CONSTRAINT university_pk PRIMARY KEY (id);

In [ ]:
## Add a SERIAL surrogate key

# -- Add the new column to the table
ALTER TABLE professors 
ADD COLUMN id serial;
________________________________
# -- Make id a primary key
ALTER TABLE professors 
ADD CONSTRAINT professors_pkey PRIMARY KEY (id);
____________________________________
# -- Have a look at the first 10 rows of professors
SELECT * FROM professors LIMIT 10;

In [ ]:
## CONCATenate columns to a surrogate key

# -- Count the number of distinct rows with columns make, model
SELECT COUNT(DISTINCT(make, model)) 
FROM cars;
________________________
# -- Add the id column
ALTER TABLE cars
ADD COLUMN id varchar(128);

# -- Update id with make + model
UPDATE cars
SET id = CONCAT(make, model);

# -- Make id a primary key
ALTER TABLE cars
ADD CONSTRAINT id_pk PRIMARY KEY(id);

# -- Have a look at the table
SELECT * FROM cars;

In [ ]:
## Test your knowledge before advancing

# -- Create the table
CREATE TABLE students (
  last_name varchar(128) NOT NULL,
  ssn integer PRIMARY KEY,
  phone_no char(12)
);

In [ ]:
## REFERENCE a table with a FOREIGN KEY

# -- Rename the university_shortname column
ALTER TABLE professors
RENAME COLUMN university_shortname TO university_id;

# -- Add a foreign key on professors referencing universities
ALTER TABLE professors 
ADD CONSTRAINT professors_fkey FOREIGN KEY (university_id) REFERENCES universities (id);

In [ ]:
## Explore Foreign Key constraints

# -- Try to insert a new professor
INSERT INTO professors (firstname, lastname, university_id)
VALUES ('Albert', 'Einstein', 'UZH');

In [ ]:
## JOIN tables linked by a foreign key

# -- Select all professors working for universities in the city of Zurich
SELECT professors.lastname, universities.id, universities.university_city
FROM professors
JOIN universities
ON professors.university_id = universities.id
WHERE universities.university_city = 'Zurich';

In [ ]:
## Add foreign keys to the "affiliations" table

# -- Add a professor_id column
ALTER TABLE affiliations
ADD COLUMN professor_id integer REFERENCES professors (id);

# -- Rename the organization column to organization_id
ALTER TABLE affiliations
RENAME organization TO organization_id;

# -- Add a foreign key on organization_id
ALTER TABLE affiliations
ADD CONSTRAINT affiliations_organization_fkey FOREIGN KEY (organization_id) REFERENCES organizations (id);


In [ ]:
## Populate the "professor_id" column

# -- Have a look at the 10 first rows of affiliations
SELECT * FROM affiliations LIMIT 10;

# -- Set professor_id to professors.id where firstname, lastname correspond to rows in professors
UPDATE affiliations
SET professor_id = professors.id
FROM professors
WHERE affiliations.firstname = professors.firstname AND affiliations.lastname = professors.lastname;

# -- Update professor_id to professors.id where firstname, lastname correspond to rows in professors
UPDATE affiliations
SET professor_id = professors.id
FROM professors
WHERE affiliations.firstname = professors.firstname AND affiliations.lastname = professors.lastname;

# -- Have a look at the 10 first rows of affiliations again
SELECT * FROM affiliations LIMIT 10;


In [ ]:
## Drop "firstname" and "lastname"

# -- Drop the firstname column
ALTER TABLE affiliations
DROP COLUMN firstname;

# -- Drop the lastname column
ALTER TABLE affiliations
DROP COLUMN lastname;

In [ ]:
## Referential integrity violations

DELETE FROM univerties WHERE id = "EPF"

In [ ]:
## Change the referential integrity behavior of a key

# -- Identify the correct constraint name
SELECT constraint_name, table_name, constraint_type
FROM information_schema.table_constraints
WHERE constraint_type = 'FOREIGN KEY';

# -- Drop the right foreign key constraint
ALTER TABLE affiliations
DROP CONSTRAINT affiliations_organization_id_fkey;

# -- Add a new foreign key constraint from affiliations to organizations which cascades deletion
ALTER TABLE affiliations
ADD CONSTRAINT affiliations_organization_id_fkey FOREIGN KEY (organization_id) REFERENCES organizations (id) ON DELETE CASCADE;

# -- Delete an organization 
DELETE FROM organizations 
WHERE id = 'CUREM';

# -- Check that no more affiliations with this organization exist
SELECT * FROM organizations
WHERE id = 'CUREM';

In [ ]:
## Count affiliations per university

# -- Count the total number of affiliations per university
SELECT COUNT(*), professors.university_id 
FROM affiliations
JOIN professors
ON affiliations.professor_id = professors.id
GROUP BY professors.university_id 
ORDER BY count DESC;

In [ ]:
## Join all the tables together

# -- Filter the table and sort it
SELECT COUNT(*), organizations.organization_sector, professors.id, universities.university_city
FROM affiliations
JOIN professors
ON affiliations.professor_id = professors.id
JOIN organizations
ON affiliations.organization_id = organizations.id
JOIN universities
ON professors.university_id = universities.id
WHERE organizations.organization_sector = 'Media & communication'
GROUP BY organizations.organization_sector, professors.id, universities.university_city
ORDER BY count DESC;